In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip ../input/aerial-cactus-identification/train.zip

In [ ]:
!unzip ../input/aerial-cactus-identification/test.zip

In [ ]:
train=pd.read_csv('../input/aerial-cactus-identification/train.csv')
train

In [ ]:
train['path']='train/'+train['id']
train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential 
from keras.layers import *
from sklearn.model_selection import StratifiedKFold #그냥 교차 검증이어도 똑같이 진행하면 된다. 

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
import glob
glob.glob('test/*')
test=pd.DataFrame({'path':glob.glob('test/*')})
test

In [ ]:
train['has_cactus']=train['has_cactus'].astype(str)
train
from efficientnet.tfkeras import EfficientNetB1
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
skf=StratifiedKFold(n_splits=5,random_state=42,shuffle=True) 
result=0
for train_index,valid_index in skf.split(train,train['has_cactus']): #원래는 트레인만 넣는거인데 정답 클래스의 비율을 맞춰서 봅기위해서 
    x_train=train.iloc[train_index] #데이터 프레임에서만(특정 데이터 접근시)/ 배열에서는 적용 안된다. 
    x_valid=train.iloc[valid_index] 
    IDG=ImageDataGenerator(horizontal_flip=True) #좌우반전해도 같은 이미지다, 학습할 때는 최대한 이미지 수 늘릴려고 
    IDG2=ImageDataGenerator() #평가나 테스트 할 때는 그대로
    IDG3=ImageDataGenerator() 
    #선언시 테스트 제너레이터에 관해서, for문 밖이라면 데이터 순서 어긋나는거 방지 10000개가 끝나고 나서 32개가 안나눠떨어짐 -> 테스트 데이터가 밀린다.배치가 밀린다. 예측 시 이상한 데이터가 들어온다. 그래서 데이터 개수랑 배치가 안맞아서 안맞아떨어질 수 있으니까
    train_generator=IDG.flow_from_dataframe(x_train,x_col="path",y_col="has_cactus",target_size=(300,300),batch_size=100) #for문 안에 안넣어주면 선언이 안된다. ID에 맞게 추출이 안된다
    valid_generator=IDG2.flow_from_dataframe(x_valid,x_col="path",y_col="has_cactus",target_size=(300,300),batch_size=100) #평가셋에도 정답 필요
    test_generator=IDG3.flow_from_dataframe(test,x_col="path",y_col=None,target_size=(300,300),batch_size=100,class_mode=None,shuffle=False) 
    model=Sequential()    
    model.add(EfficientNetB1(weights='imagenet',pooling='avg',include_top=False))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    es=EarlyStopping(patience=5)
    mc=ModelCheckpoint('best.h5',save_best_only=True)
    RL=ReduceLROnPlateau(patience=3,factor=0.2) 
    model.fit(train_generator,validation_data=valid_generator,callbacks=[es,mc,RL],epochs=3) 
    model.load_weights('best.h5') #반드시 안 쪽에 있어야만 업데이트 제대로 할 수 있슴
    result+=model.predict(test_generator)/5 #학습 5번, 테스트 5번 / 리져트를 계속 저장 how? / 그 값에다가 계속 저장/ 5개니까 평균으로 해야한다./ 교차 검증 해주면, 모든 트레인 셋을 평가셋으로 이용해줄 수 있기 때문에 점수 개선에 도움 ㅇㅋ
    #train_split->교차 검증 